In [1]:
import pandas as pd

df = pd.read_csv("../data/dataset.csv")  # or the path you used
df.head(10)



,user_id,age,budget,preferences,group_size,package_id,package_name,package_cost,package_theme,booked
0,1,25,1500,"Adventure, Beaches",2,A,Adventure in the Himalayas,1200,Adventure,0
1,1,25,1500,"Adventure, Beaches",2,B,Maldives Beach Getaway,2000,"Beach, Luxury",0
2,1,25,1500,"Adventure, Beaches",2,C,Thailand Island Hopping,1400,"Adventure, Beach",1
3,2,40,2500,"Culture, History",2,C,Thailand Island Hopping,1400,"Adventure, Beach",0
4,2,40,2500,"Culture, History",2,D,European Cultural Tour,2500,Culture,1
5,3,29,900,Wildlife,4,E,Local Safari Adventure,900,Wildlife,1
6,3,29,900,"Wildlife, Beaches",4,C,Thailand Island Hopping,1400,"Adventure, Beach",0
7,4,32,2000,"Beaches, Luxury",2,B,Maldives Beach Getaway,2000,"Beach, Luxury",1
8,4,32,2000,"Beaches, Luxury",2,D,European Cultural Tour,2500,Culture,0
9,5,27,1000,Adventure,1,A,Adventure in the Himalayas,1200,Adventure,0


In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


In [3]:
df = pd.read_csv("../data/dataset.csv")  # Adjust path if needed
df.head()


,user_id,age,budget,preferences,group_size,package_id,package_name,package_cost,package_theme,booked
0,1,25,1500,"Adventure, Beaches",2,A,Adventure in the Himalayas,1200,Adventure,0
1,1,25,1500,"Adventure, Beaches",2,B,Maldives Beach Getaway,2000,"Beach, Luxury",0
2,1,25,1500,"Adventure, Beaches",2,C,Thailand Island Hopping,1400,"Adventure, Beach",1
3,2,40,2500,"Culture, History",2,C,Thailand Island Hopping,1400,"Adventure, Beach",0
4,2,40,2500,"Culture, History",2,D,European Cultural Tour,2500,Culture,1


In [4]:
# Make a copy so we don't overwrite the original
df_encoded = df.copy()

# Create an empty set to collect every unique preference tag
all_preferences = set()

# Loop through each row's "preferences" and split by comma
for row_prefs in df_encoded['preferences']:
    # Remove any extra quotes & split by comma
    prefs_list = [p.strip().replace('"', '') for p in row_prefs.split(',')]
    for p in prefs_list:
        all_preferences.add(p)

# Convert set to a list
all_preferences = list(all_preferences)

# Initialize columns for each unique preference
for pref in all_preferences:
    df_encoded[f'pref_{pref}'] = 0

# Populate columns with 1 if the user has that preference
for idx, row in df_encoded.iterrows():
    prefs_list = [p.strip().replace('"', '') for p in row['preferences'].split(',')]
    for p in prefs_list:
        col_name = f'pref_{p}'
        df_encoded.at[idx, col_name] = 1

# Check the new columns
df_encoded.head()


,user_id,age,budget,preferences,group_size,package_id,package_name,package_cost,package_theme,booked,pref_Wildlife,pref_Culture,pref_Adventure,pref_Luxury,pref_History,pref_Beaches
0,1,25,1500,"Adventure, Beaches",2,A,Adventure in the Himalayas,1200,Adventure,0,0,0,1,0,0,1
1,1,25,1500,"Adventure, Beaches",2,B,Maldives Beach Getaway,2000,"Beach, Luxury",0,0,0,1,0,0,1
2,1,25,1500,"Adventure, Beaches",2,C,Thailand Island Hopping,1400,"Adventure, Beach",1,0,0,1,0,0,1
3,2,40,2500,"Culture, History",2,C,Thailand Island Hopping,1400,"Adventure, Beach",0,0,1,0,0,1,0
4,2,40,2500,"Culture, History",2,D,European Cultural Tour,2500,Culture,1,0,1,0,0,1,0


In [5]:
columns_to_drop = [
    'user_id',
    'package_id',
    'package_name',
    'preferences'  # We already extracted numeric features from this
]
df_encoded.drop(columns=columns_to_drop, axis=1, inplace=True)
df_encoded.head()


,age,budget,group_size,package_cost,package_theme,booked,pref_Wildlife,pref_Culture,pref_Adventure,pref_Luxury,pref_History,pref_Beaches
0,25,1500,2,1200,Adventure,0,0,0,1,0,0,1
1,25,1500,2,2000,"Beach, Luxury",0,0,0,1,0,0,1
2,25,1500,2,1400,"Adventure, Beach",1,0,0,1,0,0,1
3,40,2500,2,1400,"Adventure, Beach",0,0,1,0,0,1,0
4,40,2500,2,2500,Culture,1,0,1,0,0,1,0


In [6]:
df_encoded = pd.get_dummies(df_encoded, columns=['package_theme'])
df_encoded.head()


,age,budget,group_size,package_cost,booked,pref_Wildlife,pref_Culture,pref_Adventure,pref_Luxury,pref_History,pref_Beaches,package_theme_Adventure,"package_theme_Adventure, Beach","package_theme_Beach, Luxury",package_theme_Culture,package_theme_Wildlife
0,25,1500,2,1200,0,0,0,1,0,0,1,True,False,False,False,False
1,25,1500,2,2000,0,0,0,1,0,0,1,False,False,True,False,False
2,25,1500,2,1400,1,0,0,1,0,0,1,False,True,False,False,False
3,40,2500,2,1400,0,0,1,0,0,1,0,False,True,False,False,False
4,40,2500,2,2500,1,0,1,0,0,1,0,False,False,False,True,False


In [7]:
X = df_encoded.drop('booked', axis=1)
y = df_encoded['booked']


In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    random_state=42  # Keeps results consistent for debugging
)


In [9]:
model = LogisticRegression()
model.fit(X_train, y_train)


C:\Users\shubh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Step 7.1: Scale the Data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 7.2: Train Logistic Regression with higher max_iter
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_scaled, y_train)

# Step 7.3: Make Predictions and Evaluate
y_pred = model.predict(X_test_scaled)

acc = accuracy_score(y_test, y_pred)
print("Test Accuracy:", acc)
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Test Accuracy: 0.0

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       2.0
           1       0.00      0.00      0.00       1.0

    accuracy                           0.00       3.0
   macro avg       0.00      0.00      0.00       3.0
weighted avg       0.00      0.00      0.00       3.0



In [11]:
y_pred = model.predict(X_test)


C:\Users\shubh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [12]:
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy:", acc)


Test Accuracy: 0.6666666666666666


In [13]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.67      1.00      0.80         2
           1       0.00      0.00      0.00         1

    accuracy                           0.67         3
   macro avg       0.33      0.50      0.40         3
weighted avg       0.44      0.67      0.53         3



C:\Users\shubh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\shubh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\shubh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [14]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

# Load your CSV
df = pd.read_csv("../data/dataset.csv")  # Adjust path if needed
df.head()


,user_id,age,budget,preferences,group_size,package_id,package_name,package_cost,package_theme,booked
0,1,25,1500,"Adventure, Beaches",2,A,Adventure in the Himalayas,1200,Adventure,0
1,1,25,1500,"Adventure, Beaches",2,B,Maldives Beach Getaway,2000,"Beach, Luxury",0
2,1,25,1500,"Adventure, Beaches",2,C,Thailand Island Hopping,1400,"Adventure, Beach",1
3,2,40,2500,"Culture, History",2,C,Thailand Island Hopping,1400,"Adventure, Beach",0
4,2,40,2500,"Culture, History",2,D,European Cultural Tour,2500,Culture,1


In [15]:
# Make a copy so we don’t lose the original
df_encoded = df.copy()

# Step 2.1: Identify all unique preferences
all_preferences = set()

for row_prefs in df_encoded['preferences']:
    # Remove quotes, split on comma
    prefs_list = [p.strip().replace('"', '') for p in row_prefs.split(',')]
    for p in prefs_list:
        all_preferences.add(p)

all_preferences = list(all_preferences)  # Convert set to list

# Step 2.2: Initialize new columns for each preference
for pref in all_preferences:
    df_encoded[f'pref_{pref}'] = 0

# Step 2.3: Populate columns with 1 when the preference is present
for idx, row in df_encoded.iterrows():
    prefs_list = [p.strip().replace('"', '') for p in row['preferences'].split(',')]
    for p in prefs_list:
        df_encoded.loc[idx, f'pref_{p}'] = 1

df_encoded.head()


,user_id,age,budget,preferences,group_size,package_id,package_name,package_cost,package_theme,booked,pref_Wildlife,pref_Culture,pref_Adventure,pref_Luxury,pref_History,pref_Beaches
0,1,25,1500,"Adventure, Beaches",2,A,Adventure in the Himalayas,1200,Adventure,0,0,0,1,0,0,1
1,1,25,1500,"Adventure, Beaches",2,B,Maldives Beach Getaway,2000,"Beach, Luxury",0,0,0,1,0,0,1
2,1,25,1500,"Adventure, Beaches",2,C,Thailand Island Hopping,1400,"Adventure, Beach",1,0,0,1,0,0,1
3,2,40,2500,"Culture, History",2,C,Thailand Island Hopping,1400,"Adventure, Beach",0,0,1,0,0,1,0
4,2,40,2500,"Culture, History",2,D,European Cultural Tour,2500,Culture,1,0,1,0,0,1,0


In [16]:
columns_to_drop = [
    'user_id',       # not needed for the model
    'package_id',
    'package_name',
    'preferences'    # we already converted it into numeric columns
]

df_encoded.drop(columns=columns_to_drop, inplace=True, axis=1)
df_encoded.head()


,age,budget,group_size,package_cost,package_theme,booked,pref_Wildlife,pref_Culture,pref_Adventure,pref_Luxury,pref_History,pref_Beaches
0,25,1500,2,1200,Adventure,0,0,0,1,0,0,1
1,25,1500,2,2000,"Beach, Luxury",0,0,0,1,0,0,1
2,25,1500,2,1400,"Adventure, Beach",1,0,0,1,0,0,1
3,40,2500,2,1400,"Adventure, Beach",0,0,1,0,0,1,0
4,40,2500,2,2500,Culture,1,0,1,0,0,1,0


In [17]:
df_encoded = pd.get_dummies(df_encoded, columns=['package_theme'])
df_encoded.head()


,age,budget,group_size,package_cost,booked,pref_Wildlife,pref_Culture,pref_Adventure,pref_Luxury,pref_History,pref_Beaches,package_theme_Adventure,"package_theme_Adventure, Beach","package_theme_Beach, Luxury",package_theme_Culture,package_theme_Wildlife
0,25,1500,2,1200,0,0,0,1,0,0,1,True,False,False,False,False
1,25,1500,2,2000,0,0,0,1,0,0,1,False,False,True,False,False
2,25,1500,2,1400,1,0,0,1,0,0,1,False,True,False,False,False
3,40,2500,2,1400,0,0,1,0,0,1,0,False,True,False,False,False
4,40,2500,2,2500,1,0,1,0,0,1,0,False,False,False,True,False


In [18]:
X = df_encoded.drop('booked', axis=1)
y = df_encoded['booked']


In [19]:
X = df_encoded.drop('booked', axis=1)
y = df_encoded['booked']


In [20]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,    # 20% test, 80% train
    random_state=42   # for reproducibility
)


In [21]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [22]:
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_scaled, y_train)


LogisticRegression(max_iter=1000, random_state=42)

In [23]:
y_pred = model.predict(X_test_scaled)

acc = accuracy_score(y_test, y_pred)
print("Test Accuracy:", acc)

print("\nClassification Report:\n", classification_report(y_test, y_pred))


Test Accuracy: 0.0

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       2.0
           1       0.00      0.00      0.00       1.0

    accuracy                           0.00       3.0
   macro avg       0.00      0.00      0.00       3.0
weighted avg       0.00      0.00      0.00       3.0



In [24]:
# Step 11.1: Suppose we have user info and a specific package to evaluate
new_user = {
    "age": 30,
    "budget": 1500,
    "group_size": 2,
    # ...
    "pref_Adventure": 1,
    "pref_Beaches": 1,
    "pref_Culture": 0,
    "pref_History": 0,
    # ...
    "package_theme_Adventure": 1,  # or however your themes are
    "package_theme_Beach": 1,
    "package_cost": 1400
    # etc.
}

# Step 11.2: Turn this dictionary into a DataFrame
import pandas as pd

df_new = pd.DataFrame([new_user])

# Step 11.3: Apply the same scaling
df_new_scaled = scaler.transform(df_new)

# Step 11.4: Predict
pred = model.predict(df_new_scaled)
prob = model.predict_proba(df_new_scaled)[0][1]

print("Prediction:", pred[0])      # 1 means likely booked, 0 means not booked
print("Probability:", prob)        # Probability of booking


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- package_theme_Beach
Feature names seen at fit time, yet now missing:
- package_theme_Adventure, Beach
- package_theme_Beach, Luxury
- package_theme_Culture
- package_theme_Wildlife
- pref_Luxury
- ...


In [ ]:
new_user = {col: 0 for col in feature_columns}  # initialize everything to 0

new_user["age"] = 30
new_user["budget"] = 1500
new_user["group_size"] = 2
new_user["package_cost"] = 1400
# Preferences
new_user["pref_Adventure"] = 1
new_user["pref_Beaches"] = 1
# Themes
new_user["theme_Adventure"] = 1
new_user["theme_Beach"] = 1

df_new = pd.DataFrame([new_user])
df_new_scaled = scaler.transform(df_new)

pred = model.predict(df_new_scaled)
prob = model.predict_proba(df_new_scaled)[0][1]

print("Prediction (0=not booked, 1=booked):", pred[0])
print("Probability of booking:", prob)


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# Load your CSV
df = pd.read_csv("../data/dataset.csv")  # Adjust path if needed
df.head()


In [ ]:
df_encoded = df.copy()

# Identify all unique preferences
all_preferences = set()
for row_prefs in df_encoded['preferences']:
    prefs_list = [p.strip() for p in row_prefs.split(',')]
    for p in prefs_list:
        all_preferences.add(p)

all_preferences = list(all_preferences)  # convert set to list

# Create new columns for each unique preference
for pref in all_preferences:
    df_encoded[f'pref_{pref}'] = 0

# Populate the 1/0 flags
for idx, row in df_encoded.iterrows():
    prefs_list = [p.strip() for p in row['preferences'].split(',')]
    for p in prefs_list:
        df_encoded.at[idx, f'pref_{p}'] = 1

# Drop the original preferences column
df_encoded.drop(['preferences'], axis=1, inplace=True)

df_encoded.head()


In [ ]:
# 3.1: Collect all unique themes
all_themes = set()
for row_theme in df_encoded["package_theme"]:
    themes_list = [t.strip() for t in row_theme.split(',')]
    for t in themes_list:
        all_themes.add(t)

all_themes = list(all_themes)  # convert to list

# 3.2: Create columns like theme_Adventure, theme_Beach, etc.
for theme in all_themes:
    df_encoded[f"theme_{theme}"] = 0

# 3.3: Populate with 1 if that theme is present in the row
for idx, row in df_encoded.iterrows():
    themes_list = [t.strip() for t in row["package_theme"].split(',')]
    for t in themes_list:
        df_encoded.at[idx, f"theme_{t}"] = 1

# 3.4: Drop the original package_theme column
df_encoded.drop(["package_theme"], axis=1, inplace=True)

df_encoded.head()


In [ ]:
columns_to_drop = [
    'user_id',
    'package_id',
    'package_name'
]
df_encoded.drop(columns=columns_to_drop, axis=1, inplace=True, errors='ignore')
df_encoded.head()


In [ ]:
X = df_encoded.drop('booked', axis=1)
y = df_encoded['booked']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_scaled, y_train)


In [ ]:
y_pred = model.predict(X_test_scaled)

acc = accuracy_score(y_test, y_pred)
print("Test Accuracy:", acc)
print("\nClassification Report:\n", classification_report(y_test, y_pred))


In [ ]:
import joblib

joblib.dump(model, "voyagehack_model.pkl")
joblib.dump(scaler, "voyagehack_scaler.pkl")


In [ ]:
feature_columns = X_train.columns.tolist()
print(feature_columns)


In [ ]:
new_user = {col: 0 for col in feature_columns}  # initialize everything to 0

new_user["age"] = 30
new_user["budget"] = 1500
new_user["group_size"] = 2
new_user["package_cost"] = 1400
# Preferences
new_user["pref_Adventure"] = 1
new_user["pref_Beaches"] = 1
# Themes
new_user["theme_Adventure"] = 1
new_user["theme_Beach"] = 1

df_new = pd.DataFrame([new_user])
df_new_scaled = scaler.transform(df_new)

pred = model.predict(df_new_scaled)
prob = model.predict_proba(df_new_scaled)[0][1]

print("Prediction (0=not booked, 1=booked):", pred[0])
print("Probability of booking:", prob)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report


In [ ]:
y_pred = model.predict(X_test_scaled)  # or X_test if not using scaling


In [ ]:
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy:", acc)


In [ ]:
f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)


In [ ]:
print("Classification Report:\n", classification_report(y_test, y_pred))


In [ ]:
import joblib

# Save the trained model
joblib.dump(model, "voyagehack_model.pkl")

# Save the scaler (important if you used StandardScaler)
joblib.dump(scaler, "voyagehack_scaler.pkl")

print("Model and scaler saved successfully!")


In [ ]:
print(X_train.columns.tolist())


In [ ]:
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy:", acc)

In [ ]:
print("Classification Report:\n", classification_report(y_test, y_pred))


In [ ]:
new_user = {col: 0 for col in feature_columns}  # initialize everything to 0

new_user["age"] = 30
new_user["budget"] = 1500
new_user["group_size"] = 2
new_user["package_cost"] = 1400
# Preferences
new_user["pref_Adventure"] = 1
new_user["pref_Beaches"] = 1
# Themes
new_user["theme_Adventure"] = 1
new_user["theme_Beach"] = 1

df_new = pd.DataFrame([new_user])
df_new_scaled = scaler.transform(df_new)

pred = model.predict(df_new_scaled)
prob = model.predict_proba(df_new_scaled)[0][1]

print("Prediction (0=not booked, 1=booked):", pred[0])
print("Probability of booking:", prob)

In [ ]:
feature_columns = X_train.columns.tolist()
print(feature_columns)


In [ ]:
new_user = {col: 0 for col in feature_columns}  # initialize everything to 0

new_user["age"] = 30
new_user["budget"] = 1500
new_user["group_size"] = 2
new_user["package_cost"] = 1400
# Preferences
new_user["pref_Adventure"] = 1
new_user["pref_Beaches"] = 1
# Themes
new_user["theme_Adventure"] = 1
new_user["theme_Beach"] = 1

df_new = pd.DataFrame([new_user])
df_new_scaled = scaler.transform(df_new)

pred = model.predict(df_new_scaled)
prob = model.predict_proba(df_new_scaled)[0][1]

print("Prediction (0=not booked, 1=booked):", pred[0])
print("Probability of booking:", prob)

In [ ]:
print(X_train.head())


In [ ]:
print(X_train.columns.tolist())


In [ ]:
print(X_train.columns.tolist())

In [ ]:
print(X_train.columns.tolist())

In [ ]:
print(X_train.columns.tolist())

In [ ]:
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
print(X_train.columns.tolist())

In [ ]:
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
print("Classification Report:\n", classification_report(y_test, y_pred))

In [25]:
print(X_train.columns.tolist())

['age', 'budget', 'group_size', 'package_cost', 'pref_Wildlife', 'pref_Culture', 'pref_Adventure', 'pref_Luxury', 'pref_History', 'pref_Beaches', 'package_theme_Adventure', 'package_theme_Adventure, Beach', 'package_theme_Beach, Luxury', 'package_theme_Culture', 'package_theme_Wildlife']


In [27]:
import joblib

# Load the trained model
model = joblib.load("voyagehack_model.pkl")  # Update path if needed

# Get the exact feature names the model expects
expected_feature_order = model.feature_names_in_

# Print the expected feature order
print(list(expected_feature_order))


AttributeError: 'LogisticRegression' object has no attribute 'feature_names_in_'

In [29]:
AttributeError                            Traceback (most recent call last)
Cell In[27], line 7
      4 model = joblib.load("voyagehack_model.pkl")  # Update path if needed
      6 # Get the exact feature names the model expects
----> 7 expected_feature_order = model.feature_names_in_
      9 # Print the expected feature order
     10 print(list(expected_feature_order))

AttributeError: 'LogisticRegression' object has no attribute 'feature_names_in_'


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 7)

In [30]:
# Load your training data CSV (use the same one used to train the model)
import pandas as pd

df_train = pd.read_csv("notebooks/training_data.csv")  # Update path if needed

# Extract feature names
expected_feature_order = df_train.drop(columns=["booked"]).columns.tolist()

# Print the expected feature order
print(expected_feature_order)


FileNotFoundError: [Errno 2] No such file or directory: 'notebooks/training_data.csv'

In [31]:
# Load your training data CSV (use the same one used to train the model)
import pandas as pd

df_train = pd.read_csv("data/dataset.csv")  # Update path if needed

# Extract feature names
expected_feature_order = df_train.drop(columns=["booked"]).columns.tolist()

# Print the expected feature order
print(expected_feature_order)


FileNotFoundError: [Errno 2] No such file or directory: 'data/dataset.csv'

In [32]:
# Load the model and scaler
import joblib

model = joblib.load("voyagehack_model.pkl")  # Update path if needed
scaler = joblib.load("voyagehack_scaler.pkl")

# Get the exact feature names from the scaler (scalers store feature names)
expected_feature_order = scaler.feature_names_in_

# Print the expected feature order
print(list(expected_feature_order))


FileNotFoundError: [Errno 2] No such file or directory: 'notebooks/voyagehack_model.pkl'

In [33]:
# Load the model and scaler
import joblib

model = joblib.load("voyagehack_model.pkl")  # Update path if needed
scaler = joblib.load("voyagehack_scaler.pkl")

# Get the exact feature names from the scaler (scalers store feature names)
expected_feature_order = scaler.feature_names_in_

# Print the expected feature order
print(list(expected_feature_order))


['age', 'budget', 'group_size', 'package_cost', 'pref_Adventure', 'pref_Beaches', 'pref_History', 'pref_Culture', 'pref_Wildlife', 'pref_Luxury', 'theme_Adventure', 'theme_Beach', 'theme_Culture', 'theme_Wildlife', 'theme_Luxury']
